In [10]:
# Simplified Multimodal TFLite & TF-Keras Evaluation
import os
import pickle as pkl
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix

# Configuration
DATA_DIR = Path('../../Data/Experiment_Data/2_PreprocessDataset')
TF_MODEL_DIR = Path('../../Models/Tensorflow_Model/Multimodal')
OUTPUT_ACC_DIR = Path('../../Results/Experiment_Result/Model_Accuracy/Multimodal')
OUTPUT_PRED_DIR = Path('../../Results/Experiment_Result/Model_Preds/Multimodal')
OUTPUT_CM_DIR = Path('../../Result/Experiment_Result/Confusion_Matrix/Multimodal')
NORM_PATH = Path('../../Normalization_params/Normalization_params_pickle/normalization_params_Right_ver1.pkl')
LB_PATH = Path('../../LabelBinarizer/Label_binarizer_6_classes.pkl')

# Model names
TF_MODEL = TF_MODEL_DIR / 'MultiModal_ver1/Right/MM_Scratch.h5'

# Classes
def get_classes():
    return ['Shower','Tooth_brushing','Washing_hands','Wiping','Vacuum_Cleaner','Other']

# Utilities
def load_norm():
    return pkl.load(open(NORM_PATH,'rb'))

def load_lb():
    return pkl.load(open(LB_PATH,'rb'))

# TF-Keras batch inference
def predict_tf(pid, norm_params, lb, batch_size=256):
    model = tf.keras.models.load_model(str(TF_MODEL))
    pkl_path = DATA_DIR / pid / f'{pid}_preprocessing.pkl'
    data = pkl.load(open(pkl_path,'rb'))
    imu = data['IMU'].astype(np.float32)
    audio = data['Audio'].astype(np.float32)[...,None]
    y_true = np.array(data['Activity'])

    pm,pn,mu,sd = [norm_params[k].reshape(1,1,-1) for k in ('max','min','mean','std')]
    imu = 1 + (imu - pm)*2/(pm-pn)
    imu = (imu - mu)/sd

    imu   = np.ascontiguousarray(imu,   dtype=np.float32)
    audio = np.ascontiguousarray(audio, dtype=np.float32)

    # NumPy to Tensor
    imu_t   = tf.convert_to_tensor(imu,   dtype=tf.float32)
    audio_t = tf.convert_to_tensor(audio, dtype=tf.float32)

    preds = model.predict([imu_t, audio_t], batch_size=batch_size)
    df = pd.DataFrame(preds, columns=lb.classes_)
    df['y_true'] = y_true
    df['y_pred'] = df.drop(columns=['y_true']).idxmax(axis=1)
    return df

# Evaluation & plotting
if __name__ == '__main__':
    norm = load_norm(); lb = load_lb()
    OUTPUT_ACC_DIR.mkdir(parents=True, exist_ok=True)
    OUTPUT_PRED_DIR.mkdir(parents=True, exist_ok=True)
    OUTPUT_CM_DIR.mkdir(parents=True, exist_ok=True)

    for pid in sorted(d.name for d in DATA_DIR.iterdir() if d.is_dir()):
        print(f'Evaluating {pid}')

        # Save preds & acc
        out_pred = OUTPUT_PRED_DIR / pid
        out_pred.mkdir(exist_ok=True)

        # TF-Keras
        df_tf = predict_tf(pid, norm, lb)
        ba_tf = balanced_accuracy_score(df_tf['y_true'], df_tf['y_pred'])
        f1_tf = f1_score(df_tf['y_true'], df_tf['y_pred'], average='weighted')
        df_tf.to_csv(out_pred/f'{pid}_tf.csv', index=False)
        with open(OUTPUT_ACC_DIR/f'{pid}_tf_acc.txt','w') as f:
            f.write(f'BA: {ba_tf:.4f}\nF1: {f1_tf:.4f}')

        print(f"{pid} done: TF BA={ba_tf:.3f}")

Evaluating 201
538/538 [==============================] - 13s 24ms/step
201 done: TF BA=0.167
